In [1]:
using Graphs
using SimpleWeightedGraphs
using StatsBase: mean

In [2]:
function create_sample_graph()::SimpleGraph
    g = SimpleGraph(12)  # 12ノードのグラフ

    # コミュニティ1
    add_edge!(g, 1, 2)
    add_edge!(g, 1, 3)
    add_edge!(g, 2, 3)

    # コミュニティ2
    add_edge!(g, 4, 5)
    add_edge!(g, 4, 6)
    add_edge!(g, 5, 6)

    # コミュニティ3
    add_edge!(g, 7, 8)
    add_edge!(g, 7, 9)
    add_edge!(g, 8, 9)

    # コミュニティ間の接続
    add_edge!(g, 3, 4)
    add_edge!(g, 6, 7)
    add_edge!(g, 9, 1)

    return g
end

function create_sample_weighted_graph()::SimpleWeightedGraph
    g = SimpleWeightedGraph(12)  # 12ノードのグラフ

    # コミュニティ1
    add_edge!(g, 1, 2, 0.7)
    add_edge!(g, 1, 3, 0.6)
    add_edge!(g, 2, 3, 0.5)

    # コミュニティ2
    add_edge!(g, 4, 5, 0.3)
    add_edge!(g, 4, 6, 0.3)
    add_edge!(g, 5, 6, 0.3)

    # コミュニティ3
    add_edge!(g, 7, 8, 0.2)
    add_edge!(g, 7, 9, 0.2)
    add_edge!(g, 8, 9, 0.2)

    # コミュニティ間の接続
    add_edge!(g, 3, 4, 0.5)
    add_edge!(g, 6, 7, 0.5)
    add_edge!(g, 9, 1, 0.5)

    return g
end;

In [3]:
function fastgreedy(g::AbstractGraph)
    @show N = nv(g)
    adj_mat = adjacency_matrix(g)

    # 初期コミュニティの設定
    communities = collect(1:N)
    best_communities = copy(communities)

    # 初期モジュラリティ
    best_Q = modularity(g, communities, distmx = adj_mat)

    # 各ノードを独立したコミュニティとして扱う
    while true
        max_increase = 0.0
        max_i, max_j = 1, 1

        # すべてのペアに対してモジュラリティの増加を試す
        for i in Set(communities), j in Set(communities)
            if i < j
                j_flags = communities .== j
                communities[j_flags] .= i
                new_Q = modularity(g, communities, distmx = adj_mat)
                increase = new_Q - best_Q

                # モジュラリティの増加が最大のペアを見つける
                if increase > max_increase
                    max_increase = increase
                    max_i, max_j = i, j
                end

                # 元に戻す
                communities[j_flags] .= j
            end
        end

        # モジュラリティが改善しない場合、終了
        @show max_increase
        if max_increase <= 0.0
            break
        end

        # 最適なペアを統合
        communities[communities .== max_j] .= max_i
        best_Q += max_increase
        best_communities = copy(communities)
    end

    return best_communities
end

using Graphs
using SimpleWeightedGraphs

function weighted_clustering_coefficient(g::SimpleWeightedGraph)
    ccs = Dict{Int, Float64}()

    for u in vertices(g)
        neighbors_u = neighbors(g, u)
        degree_u = degree(g, u)

        if degree_u < 2
            ccs[u] = 0.0
            continue
        end

        local_cc = 0.0
        for i in 1:length(neighbors_u)
            for j in i+1:length(neighbors_u)
                v, w = neighbors_u[i], neighbors_u[j]
                if has_edge(g, v, w)
                    local_cc += weights(g)[v, w]
                end
            end
        end

        ccs[u] = 2 * local_cc / (degree_u * (degree_u - 1))
    end

    return ccs
end;

In [4]:
# サンプルネットワークの生成
sample_graph = create_sample_graph()
sample_weighted_graph = create_sample_weighted_graph();

In [5]:
fastgreedy(sample_graph)

N = nv(g) = 12
max_increase = 0.06249999999999999
max_increase = 0.11458333333333334
max_increase = 0.0625
max_increase = 0.11458333333333334
max_increase = 0.06249999999999997
max_increase = 0.11458333333333337
max_increase = 0.0


12-element Vector{Int64}:
  1
  1
  1
  4
  4
  4
  7
  7
  7
 10
 11
 12

In [6]:
fastgreedy(sample_weighted_graph)

N = nv(g) = 12
max_increase = 0.09895833271245166
max_increase = 0.12500001034802855
max_increase = 0.08268229177014694
max_increase = 0.048177086851663053
max_increase = 0.0512152713102598
max_increase = 0.033854164234880024
max_increase = 0.0


12-element Vector{Int64}:
  1
  1
  1
  4
  4
  4
  4
  8
  8
 10
 11
 12

In [7]:
global_clustering_coefficient(sample_graph)

0.42857142857142855

In [8]:
mean(values(weighted_clustering_coefficient(sample_weighted_graph)))

0.1527777777777778

In [9]:
g = SimpleWeightedGraph(4)

add_edge!(g, 1, 2, 0.1)
add_edge!(g, 1, 3, 0.2)
add_edge!(g, 2, 3, 0.3)
add_edge!(g, 3, 4, 0.4)

weighted_clustering_coefficient(g)

Dict{Int64, Float64} with 4 entries:
  4 => 0.0
  2 => 0.2
  3 => 0.0333333
  1 => 0.3

In [10]:
2.5 / 3

0.8333333333333334

In [40]:
function weighted_clustering_coefficient(g::SimpleWeightedGraph, i::Int)::Float64
    N = nv(g)

    max_w = maximum(g.weights)
    max_w > 0 || return 0.0

    numerator = 0.0
    denominator = 0.0

    for j in 1:N, h in 1:N
        (j == i || h == i || j >= h) && continue

        w_ij = g.weights[i, j]
        w_ih = g.weights[i, h]
        w_jh = g.weights[j, h]

        if w_ij > 0 && w_ih > 0
            harmonic_mean = (1 / w_ij + 1 / w_ih) / 2
            denominator += 2 / (harmonic_mean + 1 / max_w)
            if w_jh > 0
                numerator += 2 / (harmonic_mean + 1 / w_jh)
            end
        end
    end

    return denominator != 0 ? numerator / denominator : 0.0
end

weighted_clustering_coefficient(g::SimpleWeightedGraph)::Float64 = mean([weighted_clustering_coefficient(g, i) for i in vertices(g)])

g = SimpleWeightedGraph(12)

add_edge!(g, 1, 4, 0.8)
add_edge!(g, 1, 5, 0.7)
add_edge!(g, 2, 5, 0.6)
add_edge!(g, 2, 6, 0.5)
add_edge!(g, 3, 6, 0.4)

add_edge!(g, 4, 7, 0.7)
add_edge!(g, 5, 8, 0.6)
add_edge!(g, 6, 9, 0.5)

add_edge!(g, 7, 10, 0.4)
add_edge!(g, 8, 11, 0.3)
add_edge!(g, 9, 12, 0.2)

add_edge!(g, 10, 11, 0.6)
add_edge!(g, 11, 12, 0.7)
add_edge!(g, 12, 10, 0.8)

for i in 1:12
    println("$(i):\t$(weighted_clustering_coefficient(g, i))")
end

weighted_clustering_coefficient(g)

1:	0.0
2:	0.0
3:	0.0
4:	0.0
5:	0.0
6:	0.0
7:	0.0
8:	0.0
9:	0.0
10:	0.3501627358868705
11:	0.39737359114879534
12:	0.396616432926886


0.09534606333021266

0.09534606333021266